<a href="https://colab.research.google.com/github/paruliansaragi/Notebooks/blob/master/cs231n_Optimization%2C_Derivatives_and_Backprop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Optimization

http://cs231n.github.io/optimization-1/



Optimization is the process of finding the set of $W$ that minimizes the loss.

The loss functions are hard to visualize as they are over high-dimensional spaces. 

We can slice a plane or ray to get a 1 or 2-D view. We can generate a random $W_1$ and compute the loss along this direction by evaluating $L(W + aW_1)$ for different value of $a$. This plots $a$ as the x-axis and the loss as y. 

Lets explain the piecewise-linear structure of the loss by the math:

$L_i = \sum_{j\neq y_i} \left[ \max(0, w_j^Tx_i - w_{y_i}^Tx_i + 1) \right]$ 

From the equation the data loss for each example is a sum of (zero threshold) linear functions of W. Each row of $W$ has positive when its the wrong class and negative when correct class. Consider a dataset that has 3-D points and three classes. Full SVM loss w/out regularisation:

$\begin{align}
L_0 = & \max(0, w_1^Tx_0 - w_0^Tx_0 + 1) + \max(0, w_2^Tx_0 - w_0^Tx_0 + 1) \\\\
L_1 = & \max(0, w_0^Tx_1 - w_1^Tx_1 + 1) + \max(0, w_2^Tx_1 - w_1^Tx_1 + 1) \\\\
L_2 = & \max(0, w_0^Tx_2 - w_2^Tx_2 + 1) + \max(0, w_1^Tx_2 - w_2^Tx_2 + 1) \\\\
L = & (L_0 + L_1 + L_2)/3
\end{align}$

Since these examples are 1-D, the data $x_i$ and weights $w_j$ are numbers. 

![alt text](http://cs231n.github.io/assets/svmbowl.png)

The data loss is the sum of multiple terms. The loss is bowl-shaped (convex). NN's tend to produce non-convex. Non-differentiable loss functions: you can see that the kinks in the loss function (due to the mx operation) make the loss non-differentiable because at these kinks the gradient is not defined. However, the subgradient still exists and is used instead. 

The goal is to find weights to minimize the loss. 

Strategy 1# Random Search and keep track of what works best?

```
bestloss = float("inf") # python assigns highest float value to inf
for n in range(10000):
W = np.random.randn(10, 3073) * 0.0001
loss = L(X_train, Y_train, W)
if loss < best_loss
bestloss=loss
bestW=W
```

15% accuracy.... **Core idea: iterative refinement:** finding the best set of weights $W$ is near impossible but refining a set of weights to be better is much less difficult. 

Strategy 2# Random Local Search

Strategy #3 Following the Gradient

Gradient : vector of partial derivatives. The gradient provides the direction of steepest ascent (if you do the dot product of the unit vector of that gradient with gradient). If we simply do 1 - the direction of steepest ascent we get the direction of steepest descent. Therefore, if we compute the best direction along which we should change our weights that is the direction of steepest descent. 

For 1-D (single variable) functions, the slope is the instantaneous rate of change of the function at any point. The gradient then is the slope for functions that take a vector of numbers (multivariable functions, therefore we compute the partial derivatives). 
The equation for the derivative:

$\frac{df(x)}{dx} = \lim_{h\ \to 0} \frac{f(x + h) - f(x)}{h}$

###Computing the gradient

There are 2 ways to compute the gradient: A slow, approximate but easy way (**numerical gradient**), and a fast, exact but more error prone way that requires calculus (**analytic gradient**). 

####Computing the gradient numerically with finite differences

The above formula is the numerical gradient. Below is a function that takes a function f and a vector x to evaluate the gradient on and returns the gradient of f at x:

```
def numeric_grad(f, x):
"""
- f : function with single arg
- x : points (np array) to evaluate the gradient at
"""
fx = f(x)
grad = np.zeros(x.shape)
h = 0.00001 # small value to nudge by
#iterate over all indexes in x
it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
while not it.finished:
ix = it.multi_index
old_value = x[ix]
x[ix] = old_valute + h #inc by h
fxh = f(x) # eval f(x + h)
x[ix] = old_value #resture to prev value
#compute partial derivative
grad[ix] = (fxh-fx) / h #the slope
it.iternext() # step to next dimension
```

The code iterates over all dimensions one by one, makes a small change i.e. h along that dimension and calculates the partial derivative of the loss function along that dimension by seeing how much that function changed.

Mathematically the formula of the gradient requires the limit of h to go to 0, but in practice it is sufficient to be a very small number (1e-5).
**NB:**in practice it is often better to compute the numeric gradient using the centered difference formula : $[f(x+h) - f(x-h)] / 2 h$

We can use the function above to compute the gradient at any point for any function. 

```
def cifar10_loss(W):
return L(X_train, Y_train, W)

W = np.random.randn(10, 3073) * 0.001
df = numeric_grad(cifar10_loss, W)
```
We can now use the gradient to update the weights
```
loss_old = cifar10_loss(W)
for stepsize in [-10, -5, -1]:
step_size = 10 ** stepsize
W_new = W - step_size * df
loss_new = cifar10_loss(W_new)
print('for step size %f new loss: %f' % (step_size, loss_new))
```
As mentioned we compute the new weights in the negative of the gradient because the gradient shows you the direction of steepest ascent and we just go in the opposite of that direction.

The step size dictated by the learning rate is one of the most important hyperparameters (there are several techniques such as learning rate scheduling/annealing etc..). 

Evaluating the numerical gradient has a linear complexity in the number of parameters. We had 30730 params and thus had to perform 30,731 evaluations of the loss to evaluate the gradient and to perform a single parameter update. 

####Computing the gradient analytically with Calculus

The numerical gradient is very simple to compute using the finite difference approximation, the downside is that it is approximate (we pick a small value of h, while the true gradient is as h approaches 0), and it is computationally expensive to compute. The second way is using calculus, allowing us to derive a direct formula for the gradient (no approximations) that is also very fast to compute. It can be more error prone to implement, that is why its common to compute the analytic gradient and compare it to the numerical gradient to check the correctness. This is called a **gradient check**.

Usings the SVM loss for a single datapoint:

$L_i = \sum_{j\neq y_i} \left[ \max(0, w_j^Tx_i - w_{y_i}^Tx_i + \Delta) \right]$

We can differentiate the function w.r.t to the weights. E.g. taking the gradient w.r.t $w_yi$ we get: 

$\nabla_{w_{y_i}} L_i = - \left( \sum_{j\neq y_i} \mathbb{1}(w_j^Tx_i - w_{y_i}^Tx_i + \Delta > 0) \right) x_i$

where 1 is the indicator function (that is one if the condition inside is true or 0 otherwise). This is simply counting the number of classes that didn't meet the desired margin (and hence contributed to the loss function) and then the data vector $x_i$ scaled by this number is the gradient. This is the gradient only w.r.t the row of $W$ that corresponds to the correct class. For rows where ${j\neq y_i}$ the gradient is:

$\nabla_{w_j} L_i = \mathbb{1}(w_j^Tx_i - w_{y_i}^Tx_i + \Delta > 0) x_i$

###Gradient Descent

Now we have the gradient of the loss, we can simply repeatedly evaluate the gradient and update the parameters. 

```
while True:
weights_grad = evaluate_gradient(loss_fun, data, weights)
weights += - step_size * weights_grad # param update
```
This is at the core of NN libraries.

**Mini-batch gradient descent:** It is wasteful to compute the full loss function over the entire training set to perform a single param update. It is better to compute the gradient over batches of data. We use batches to perform param updates:
```
while True:
data_batch = sample_training_data(data, batch_size=256)
weights_grad = eval_grad(loss_fn, data_batch, weights)
weights += - step_size * weights_grad
```

This works well when we have copies of images and we'd get the same loss so we'd be wasting computation. The mini-batch is a good approximation of the full and the gradient of the objective function is a good approximation as a result. We can achieve convergence much quicker. It is then better to perform more updates with fewer samples.

The extreme case where the mini-batch contains only one example is called stochastic gradient descent. Sometimes people use the term SGD to mean Mini-batch GD. 







#Derivatives, Backprop and Vectorization


###Derivatives
####Scalar Case
Derivatives help us measure change, that is if we were to change variable x by h what is the resultant change in y. The **chain rule** tells us how to compute the derivative of the composition of functions. Say function z = g(f(x)). If we wanted to compute the effect of x on z, if x changes by hx then y will change by the derviative of x and y = the derivative of y w.r.t x. If y changes by hy then z will change by the derivative of z w.r.t y. 

So we take the derivative of the outer function then multiply by the derivative of the inner. 

Say $\frac{d}{dx}[ln(sin(x))]$ = $f'(g(x))$ = $f' (g(x)) . g' (x)$

Basically, the chain rule says we have to take the derivative of the outer function w.r.t the inner function. 

$f'(g(x)) = \frac{1}{sin(x)}$ then we multiply this with the inner

$g'(x) = cos(x)$ so the derivative is $\frac{1}{sin(x)} . cos(x)$.

So the chain rule is just finding the deriviative of composite functions. 

The chain rule says:
$\frac{d}{dx}[f(g(x))] = f'(g(x)) . g'(x)$

Our function is composite because it is a function within a function. 
Say $f(x) cos(x)$ and $g(x) = x^2$ then $cos(x^2) = f(g(x)).$ g is the inner function and f the outer.

Say now that $g(x) = 5- 6x$ inner function and $f(x) = x^5$ outer. We can use the chain rule: $\frac{d}{dx}[f(g(x))] = f'(g(x)) . g'(x)$
Lets find the derivatives of the inner and outer functions: 
$g'(x) = -6$ and $f'(x) = 5x^4$. That means:
$5(5-6x)^4 . -6$ remember the first part of the expression is the inner and outer.
$=-30(5-6x)^4$

####Jacobian & Generalized Jacobian: Tensor in Tensor out

It is the M x N matrix of partial derivatives. It tells us the amount by which $y_i$ will change if $x_j$ changed by a small amount. 

In deep learning we take tensors as inputs and tensors as outputs. The generalized jacobian tells the same relationship but for a tensor. 

####Backprop with Tensors

A layer f is a function of tensor inputs x and weights w; the tensor output of the layer is then y = f(x, w). The layer f has a scalar loss L. During backprop we are given the $\frac{dL}{dy}$ and want to compute $\frac{dL}{dx}$ and $\frac{dL}{dw}$. We know from the chain rule that: $\frac{dL}{dx}= \frac{dL}{dy} . \frac{dy}{dx}$ and  $\frac{dL}{dw}= \frac{dL}{dy} . \frac{dy}{dw}$. However, the jacobian is too large to fit in memory. Say f is a linear layer and takes a minibatch N vectors each of D dimension, and produces mini-batch of N vectors each of dimension M. Then x is N x D, w is D x M and y = f(x,w) = xw is a matrix of shape N x M. 

The Jacobian then has shape (N x M) x (N x D). A typical NN may have N = 64, and M = D = 4096; then  $\frac{dy}{dx}$ consists of 64 * 4096 * 64 * 2096 scalar values; which is > 68 billion; using 32-bit floating point, this jacobian matrix will take 256gb of memory. 

However, it turns out that for most common NN layers, we can derive expressions that compute the product  $\frac{dy}{dx}\frac{dL}{dy}$ without forming the jacobian. Even better, we can derive this without computing an explicit expression for the jacobian. 

Say N = 1, D=2, M=3.
$y = (y_{1,1}, y_{1,2}, y_{1,3}) = xw$
$= (x_{1,1}, x_{1,2}) (w_{1,1}, w_{1,2}, w_{1,3}
w_{2,1}, w_{2,2}, w_{2,3})T$
$=(x_{1,1}w_{1,1}+x_{1,2}w_{1,2}
x_{2,1}w_{2,1}+x_{2,2}w_{2,2}
x_{3,1}w_{3,1}+x_{3,2}w_{3,2})$
During backprop we assume we have access to $\frac{dL}{dy}$ which has shape (1) x (N x M) then we can write:
$\frac{dL}{dy} = (dy_{1,1}, dy_{1,2}, dy_{1,3})$
Our goal is to derive an expression for $\frac{dL}{dx}$ in terms of x, w, and  $\frac{dL}{dy}$ without forming the entire jacobian. We know each element of  $\frac{dL}{dx}$ (jacobian) is a scalar giving the partial derivatives of L w.r.t the elements of x:

 $\frac{dL}{dx} = (\frac{dL}{dx_{1,1}} \frac{dL}{dx_{1,2}})$
 
 Thinking in terms of one element at a time the chain rule tells us that:
 
$ \frac{dL}{dx_{1,1}} = \frac{dL}{dy} \frac{dy}{dx_{1,1}}$

$ \frac{dL}{dx_{1,2}} = \frac{dL}{dy} \frac{dy}{dx_{1,2}}$

If we view $\frac{dL}{dy}$ and $\frac{dy}{dx_{1,1}}$ as matrices of shape N x M, then their generalized matrix product is simply the dot product $ \frac{dL}{d .y} \frac{dy}{dx_{1,1}}$.
Now we compute:

$\frac{dy}{dx_{1,1}} = (\frac{dy_{1,1}}{dx_{1,1}} \frac{dy_{1,2}}{dx_{1,1}}, \frac{dy_{1,3}}{dx_{1,1}} ) = (w_{1,1} w_{1,2} w_{1,3})$

$\frac{dy}{dx_{1,2}} = (\frac{dy_{1,1}}{dx_{1,2}} \frac{dy_{1,2}}{dx_{1,2}}, \frac{dy_{1,3}}{dx_{1,2}} ) = (w_{2,1} w_{2,2} w_{2,3})$

This gives the final expression for $\frac{dL}{dx}$:

$\frac{dL}{dx} = (\frac{dL}{dx_{1,1}} \frac{dL}{dx_{1,2}})$

$=(dy_{1,1}w_{1,1} + dy_{1,2}w_{1,2} + dy_{1,3}w_{1,3}$
$dy_{1,1}w_{2,1} + dy_{1,2}w_{2,2} + dy_{1,3}w_{2,3})^{T}$

$=\frac{dL}{dy}x^{T}$

This final result is interesting because it allows us to efficiently compute the $\frac{dL}{dx}$ without forming the jacobian. 


NB: the jacobian has a nice property called local linearity. Zooming in on a point the neighbourhood around the specific point looks linear. 



#Backprop
http://cs231n.github.io/optimization-2/


Backprop : a way of computing gradients of expressions through recursive application of the chain rule. 

The core problem in this section is as follows: we have some function f(x) where x is a vector of inputs and we want to compute the gradient of f at x(i.e. $\nabla f(x)$)

Lets consider: $f(x,y,z)=(x+y)z$. This can be broken down into $q=x+y$ and $f=qz$. f is just a multiplication of q and z so, $\frac{\partial f}{\partial q} = z, \frac{\partial f}{\partial z} = q$ and q is the addition of x and y so $\frac{\partial q}{\partial x} = 1, \frac{\partial q}{\partial y} = 1$. We dont care about the gradient of the q. We want the gradient of f w.r.t its inputs x,y,z. The chain rule tells us the correct way to chain these gradient expression is through multiplication. That is $\frac{\partial f}{\partial x} = \frac{\partial f}{\partial q} \frac{\partial q}{\partial x}$:

```
x = -2; y = 5; z = -4
# perform the forward pass
q = x + y # q becomes 3
f = q * z # f becomes -12

# perform the backward pass (backpropagation) in reverse order:
# first backprop through f = q * z
dfdz = q # df/dz = q, so gradient on z becomes 3
dfdq = z # df/dq = z, so gradient on q becomes -4
# now backprop through q = x + y
dfdx = 1.0 * dfdq # dq/dx = 1. And the multiplication here is the chain rule!
dfdy = 1.0 * dfdq # dq/dy = 1
```

We are left with the gradients [dfdx, dfdy, dfdz], which tells us the sensitivty of the variables x,y,z on f. 

####Intuition of backprop

Backprop is a local process, every gate in a circuit diagram gets some inputs and compute two things: 1. its output value and 2. the local gradient of its inputs w.r.t its output value. They do this without awareness of any other details of the full circuit. Once the forward pass is over it will learn about the gradient of its output value on the final output of the entire circuit. The chain rule says take that gradient and multiply it into every gradient for all its inputs.

![alt text](https://xiandong79.github.io/downloads/circuit_diagram.png)

The add gate receives an input -2, 5 and computes output 3. Since its computing addition, its local gradient for both of its inputs is +1. The rest of the circuit computes the final value -12. During the backward pass, the chain rule is applied recursively through the circuit, the add gate (which is an input to the multiply gate) leans that the gradient for its output was -4. The circuit wants a higher value, then the circuit wants the output of the add gate to be lower (due to the negative sign) and with force of 4. The add gate takes that gradient and multiplies it to all of the local gradients for its inputs (making the gradient on both x and y 1 * -4 = -4). This has the desired effect, if x,y were to decrease (in response to their negative gradient) then the add gate's output would decrease, in turn making the multiply gate's output increase. 

Backprop = gate's talking to each other.

####Signmoid example
Any differentiable function can be a gate, we can group multiple gates into a single gate, or decompose a function into multiple gates. 

Example: $f(w,x) = \frac{1}{1+e^{-(w_0x_0 + w_1x_1 + w_2)}}$

This is a 2-D neuron with inputs x and weights w that uses a sigmoid activation function. The function comprises multiple gates:

$f(x) = \frac{1}{x} 
\hspace{1in} \rightarrow \hspace{1in} 
\frac{df}{dx} = -1/x^2 
\\\\
f_c(x) = c + x
\hspace{1in} \rightarrow \hspace{1in} 
\frac{df}{dx} = 1 
\\\\
f(x) = e^x
\hspace{1in} \rightarrow \hspace{1in} 
\frac{df}{dx} = e^x
\\\\
f_a(x) = ax
\hspace{1in} \rightarrow \hspace{1in} 
\frac{df}{dx} = a$

Where functions $f_c, f_a$ translate the input by a constant c and scale the input by constant a. These are special cases of addition and multiplication, they are new unary gates, since we dont need the gradients for c,a. 

![alt text](https://xiandong79.github.io/downloads/circuit.png)

We see above a long chain of functions that operates on the result of the dot product of w,x. This is the sigmoid function. The derivative of the sigmoid w.r.t to its input simplifies if you perform the derivation (after adding and subtracting 1 in the numerator):

$\sigma(x) = \frac{1}{1+e^{-x}} \\\\
\rightarrow \hspace{0.3in} \frac{d\sigma(x)}{dx} = \frac{e^{-x}}{(1+e^{-x})^2} = \left( \frac{1 + e^{-x} - 1}{1 + e^{-x}} \right) \left( \frac{1}{1+e^{-x}} \right) 
= \left( 1 - \sigma(x) \right) \sigma(x)$

The gradient simplies nicely. E.g. the sigmoid recieves 1.0 as input and computes the output as 0.73. The derivation shows the local gradient would simply be (1 - 0.73) * 0.73 which is roughly 0.2. In practice its good to group these operations into a single gate. 

```
w = [2,-3,-3] # assume some random weights and data
x = [-1, -2]

# forward pass
dot = w[0]*x[0] + w[1]*x[1] + w[2]
f = 1.0 / (1 + math.exp(-dot)) # sigmoid function

# backward pass through the neuron (backpropagation)
ddot = (1 - f) * f # gradient on dot variable, using the sigmoid gradient derivation
dx = [w[0] * ddot, w[1] * ddot] # backprop into x
dw = [x[0] * ddot, x[1] * ddot, 1.0 * ddot] # backprop into w
# we're done! we have the gradients on the inputs to the circuit
```

###Backprop in practice

Another example: 

$f(x,y) = \frac{x + \sigma(y)}{\sigma(x) + (x+y)^2}$

Its good to keep track of the intermediate variables. So cache forward pass variables. 

![alt text](https://xiandong79.github.io/downloads/circuit2.png)

Looking at the above we can see:

The add gate always takes the gradient on its output and distributes it equally to all of its inputs. The local gradient for the add op is simply +1.0 so the gradients on all inputs will exactly equal the gradients on the output because it is multiplied by x1.0. Note that the + gate routed the gradient of 2.00 to both of its inputs, equally and unchanged. 

The max gate routes the gradient. The max gate distributes the gradient unchanged to one of its input (the one with the highest value during forward pass). This is because the local gradient for a max gate is 1.0 for the highest value and 0 for all others. In the above, the max routes 2.00 grad to z. 

The multiply gate: its local gradients are the input values (except switched) which is multiplied by the gradient on its output during the chain rule. The gradient on x is -8.00, which is -4.00 x 2.00. 

Notice: if one input to the multiply gate is very small and other is very big then the multiply gate will do something unintuitive: it will asign a huge gradient to the small and a tiny to the large. In linear classifier where the weights are dot producted with inputs, this implies the scale of the data has an effect on the magnitude of the gradient for the weights. E.g. if you multiplied all input data by 1000 during preprocessing then the grad on the weights will be 1000x larger, and you'd have a lower learning rate.

###Gradients for vectorized ops

The concepts above extend to matrix/vector ops. 

**Matrix-Matrix multiply gradient**. The most tricky op is mat-mat multipication:

```
W = np.random.randn(5, 10)
X = np.random.randn(10, 3)
D = W.dot(X)

# now suppose we had the gradient on D from above in the circuit
dD = np.random.randn(*D.shape) # same shape as D
dW = dD.dot(X.T) 
dX = W.T.dot(dD)

```

For instance, we know that the gradient on the weights dW must be of the same size as W after it is computed, and that it must depend on matrix multiplication of X and dD (as is the case when both X,W are single numbers and not matrices). There is always exactly one way of achieving this so that the dimensions work out. For example, X is of size [10 x 3] and dD of size [5 x 3], so if we want dW and W has shape [5 x 10], then the only way of achieving this is with dD.dot(X.T), as shown above.




http://cs231n.stanford.edu/vecDerivs.pdf

Shows how the derivative of y w.r.t x is just the dot product of the weights and the input. Since the y is a summation of the dot products of weights and inputs. The derivative 
$\frac{\partial y_3}{\partial x_7} = \frac{\partial}{\partial x_7}[W_{3,1}x_1 + W_{3,2}x_2 + ... + W_{3,D}x_D]$

Since none of the other terms in the summation include $x_7$ their derivatives w.r.t to $x_7$ are all 0. Thus:

$= 0 + 0 + .. +  \frac{\partial}{\partial x_7}[W_{3,7}x_7]+...+0$

$=\frac{\partial}{\partial x_7}[W_{3,7}x_7]$
$=W_{3,7$

So the vector of y's is just equal to $Wx$.
If you do the same process you will find that for all i and j:

$\frac{\partial y_i}{\partial x_j} = W_{i,j}$ 

Thus the matrix of partial derivatives (Jacobian) is equivalent to your weight matrix $W$. Thus we can conclude that:

$\vec{y}=W\vec{x}$ and we have $\frac{d \vec{y} } {d \vec{x}}$
